In [ ]:
#! lamin init --storage ./lamin-intro --schema bionty 

import scanpy as sc
from scprint import scPrint
from scdataloader import Preprocessor
from scdataloader.utils import load_genes
import numpy as np
import anndata as ad
from huggingface_hub import hf_hub_download

from scprint.tasks import Denoiser, withknn
from scprint.model.utils import downsample_profile
from scipy.stats import spearmanr
from anndata import AnnData

import seaborn as sns
%load_ext autoreload
%autoreload 2

import torch
torch.set_float32_matmul_precision('medium')

→ connected lamindb: jkobject/scprint2


In [ ]:
#model_checkpoint_file = hf_hub_download(
#    repo_id="jkobject/scPRINT", filename=f"v2-medium.ckpt"
#)
#model_checkpoint_file = ../data/
model_checkpoint_file = '../../../80vhz9uh.ckpt'

In [ ]:
m = torch.load(model_checkpoint_file)
if "prenorm" in m['hyper_parameters']:
    m['hyper_parameters'].pop("prenorm")
    torch.save(m, model_checkpoint_file)
if "label_counts" in m['hyper_parameters']:
    model = scPrint.load_from_checkpoint(model_checkpoint_file, precpt_gene_emb=None, classes=m['hyper_parameters']['label_counts'])
else:
    model = scPrint.load_from_checkpoint(model_checkpoint_file, precpt_gene_emb=None)

In [ ]:
model = scPrint.load_from_checkpoint(model_checkpoint_file, precpt_gene_emb=None)

FYI: scPrint is not attached to a `Trainer`.


In [ ]:
stdata = stdata[:, ["BLANK" not in i and "NegControl" not in i for i in stdata.var.index.tolist()]]

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(15, 4))

axs[0].set_title("Total transcripts per cell")
sns.histplot(
    stdata.obs["total_counts"],
    kde=False,
    ax=axs[0],
)

axs[1].set_title("Unique transcripts per cell")
sns.histplot(
    stdata.obs["n_genes_by_counts"],
    kde=False,
    ax=axs[1],
)

In [ ]:
# spec xenium 
stdata = stdata[:, ["BLANK" not in i and "NegControl" not in i for i in stdata.var.index.tolist()]]

# set to 300, 1 for visium
sc.pp.filter_cells(stdata, min_counts=50, )
sc.pp.filter_genes(stdata, min_cells=5)

stdata.layers["counts"] = stdata.X.copy()
sc.pp.normalize_total(stdata, inplace=True)
sc.pp.log1p(stdata)
sc.pp.pca(stdata)
sc.pp.neighbors(stdata)
sc.tl.umap(stdata)
sc.tl.leiden(stdata)

In [ ]:
sq.pl.spatial_scatter(
    stdata,
    library_id="spatial",
    shape=None,
    color=[
        "leiden",
        "total_counts",
    ],
    wspace=0.4,
    ncols=1,
)

In [ ]:
stdata = stdata[stdata.obs.leiden != "0", :]
stdata = stdata.copy()
stdata

In [ ]:
# preprocess

In [ ]:
embed = Embedder(how="most var", max_len=500, add_zero_genes=0, num_workers=16, pred_embedding = ["cell_type_ontology_term_id"], keep_all_cls_pred=False, output_expression="none", batch_size=64)
